# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/GenAI/RAG/Knowledge Graphs with LightRAG

/content/drive/MyDrive/GenAI/RAG/Knowledge Graphs with LightRAG


In [2]:
!pip install PyPDF2 lightrag-hku pyvis aioboto3 ollama tiktoken nano_vectordb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00


In [4]:
# import libraries
import os
from IPython.display import Markdown, display
from lightrag import LightRAG, QueryParam
from lightrag.llm import gpt_4o_mini_complete, gpt_4o_complete
import PyPDF2
from google.colab import userdata

In [5]:
# Set the OpenAI API key in the environment
api_key = userdata.get('genai_course')
os.environ['OPENAI_API_KEY'] = api_key

# Create a directory

In [19]:
# Set the name for the directory
WORKING_DIR = "./working_dir"

if not os.path.exists(WORKING_DIR):
    os.makedirs(WORKING_DIR)

rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func= gpt_4o_complete
)

In [7]:
# If you are using Colab / Jupyter, do this:
import nest_asyncio
nest_asyncio.apply()

# Data Preparation

* PDF => TXT
* Create RAG with TXT

In [8]:
# Open the PDF and extract the text
pdf_text = []
with open("Greek-gods-story.pdf", "rb") as file:
  reader = PyPDF2.PdfReader(file)
  for page in reader.pages:
    pdf_text.append(page.extract_text())

In [10]:
# Save (Write) a file with the data
with open("greek_gods.txt", "w") as file:
  for line in pdf_text:
    file.write(f"{line}\n")

In [11]:
# Build the RAG
with open("greek_gods.txt", "r") as file:
  rag.insert(file.read())

Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.29s/batch]

Level 2 - Extracting entities and relationships:   0%|          | 0/3 [00:00<?, ?chunk/s]
Level 2 - Extracting entities and relationships: 100%|██████████| 3/3 [00:33<00:00, 11.06s/chunk]
                                                                                                 

Level 3 - Inserting entities:   0%|          | 0/38 [00:00<?, ?entity/s]

                                                                        


Level 3 - Inserting relationships:   0%|          | 0/17 [00:00<?, ?relationship/s]


                                                                                   
Generating embeddings: 100%|██████████| 2/2 [00:03<00:00,  1.63s/batch]

Processing batch 1: 100%|██████████| 1/1 [00:41<00:00, 41.50s/it]


# RAG

In [20]:
# PErform Naive RAG
def perform_rag(query, mode): # Changed function name to avoid shadowing
  return Markdown(rag.query(query, # Now rag refers to the LightRAG object
                            param = QueryParam(mode = mode)))
query = "Who is Hermes"
perform_rag(query, "naive") # Call the new function name

Hermes is a Greek god known for his role as a messenger between the gods and mortals, which he carried out with speed and efficiency thanks to his winged hat and sandals. He is a son of Zeus, the king of the gods. 

### Birth and Early Adventures
Hermes' story begins dramatically; he was born in a cave, and only hours after his birth, he created the lyre, a musical instrument, by using a tortoise shell and strings. This early act of ingenuity marks the beginning of his reputation for cleverness and resourcefulness.

### The Mischievous Thief
Despite being a baby, Hermes quickly showcased his mischievous nature by stealing a herd of cattle belonging to Apollo, another god. His cunning allowed him to hide the cattle effectively, returning to his mother without arousing suspicion. When Apollo eventually confronted him, Hermes managed to charm his way out of trouble by playing the lyre. Enchanted by the music, Apollo allowed Hermes to keep the cattle in exchange for the lyre.

### Role and Reputation
Throughout his life, Hermes maintained his mischievous persona. His ability to travel quickly made him an ideal messenger, a role he embodied throughout his mythology. His cunning and wit made him a beloved figure in Greek mythology, particularly in tales that highlight his clever solutions to problems and his trickster nature.

In [21]:
# Perform LightRAG with a local search
# This means we look into closer entities
perform_rag(query, "local")

```json
{
  "high_level_keywords": ["Hermes", "Mythology", "Greek gods"],
  "low_level_keywords": ["Messenger of the gods", "God of trade", "Son of Zeus", "Symbol: Caduceus", "Hermes myths"]
}
```


Hermes is a prominent figure in Greek mythology, best known as the son of Zeus, the king of the gods. He is characterized by his cleverness, inventiveness, and role as a messenger among gods and mortals. 

### Birth and Attributes
Hermes was born in a cave and is famous for his precociousness. Shortly after his birth, Hermes created the lyre, a stringed musical instrument, by stringing chords across a tortoise shell. This early display of ingenuity is a hallmark of his personality in mythology. Hermes also donned a winged hat and sandals, which allowed him to travel quickly, facilitating his role as a divine messenger.

### Relationship with Other Gods
Hermes had significant interactions with other deities. For instance, he once stole cattle from Apollo but later reconciled by giving Apollo the lyre, thus establishing a complex and reciprocal relationship with him. His father, Zeus, often traveled with Hermes, disguised as ordinary men, which led to various mythological narratives involving hospitality and divine intervention.

### Role and Significance
In his capacity as a messenger, Hermes is vital in conveying messages between gods and humans. This role highlights his essential position in the pantheon as a connector and communicator. Additionally, Hermes' reputation for mischief and quick wit adds to his unique identity among the gods, making him a versatile figure in mythological stories.

Through these narratives, Hermes is portrayed not only as a divine trickster and communicator but also as an instrumental figure in various mythological events and relationships.

In [22]:
# Perform LightRAG with Global Search
# More diverse results, global entity relationships
perform_rag(query, "global")

```json
{
  "high_level_keywords": ["Hermes", "Mythology"],
  "low_level_keywords": ["Greek god", "Messenger", "Travelers", "Commerce", "Thieves"]
}
```


### Overview of Hermes

Hermes is a prominent figure in Greek mythology, known for his distinct personality and versatile roles. He is identified as the son of Zeus, the king of the gods. Hermes is celebrated for his cleverness, inventiveness, and dexterity, often depicted as a helper and messenger among the gods and mortals. His unique attributes and adventures make him a multifaceted character within the pantheon.

### Attributes and Abilities

Hermes is renowned for his resourcefulness and creativity. One of his notable inventions is the lyre, which he crafted using a tortoise shell. This act of creation is illustrative of his knack for inventiveness and his connection to music and the arts. Furthermore, Hermes possesses a winged hat and sandals, allowing him to move swiftly, which aids him in his role as the messenger of the gods.

### Mythological Stories Involving Hermes

One popular story involves Hermes' childhood antics and his relationship with Apollo, another prominent god. As a newborn, Hermes exhibited his cunning nature by stealing Apollo's cattle. To appease Apollo, Hermes gifted him the lyre, establishing a complex relationship between the two gods. This tale highlights Hermes' mischievous yet diplomatic characteristics.

Another significant narrative includes his journey with his father, Zeus. Disguised as mortals, they traveled seeking hospitality and were welcomed by the elderly couple Baucis and Philemon. Their experience with this compassionate couple led to lasting rewards for Baucis and Philemon, showcasing Hermes' role in divine intervention and reward.

### Role Among the Gods

Hermes serves as a critical link between the divine and mortal realms, utilizing his speed and wit to deliver messages and guide souls to the underworld. These responsibilities underscore his importance in maintaining communication and order among the gods and humans, reflecting his adaptability and significance within Greek mythology.

In [23]:
# Perform LightRAG with Hybrid approach
perform_rag(query, "hybrid")

```json
{
  "high_level_keywords": ["Hermes", "Mythology", "Greek gods"],
  "low_level_keywords": ["Messenger", "Winged sandals", "Caduceus", "Zeus", "Olympus"]
}
```


Hermes is a prominent figure in Greek mythology known for his multifaceted roles and attributes. As the son of Zeus, Hermes is renowned for his cleverness, inventiveness, and function as a messenger among gods and mortals. His clever nature is highlighted by his creation of the first musical instrument, the lyre, which he crafted from a tortoise shell. This act of creativity reflects his inventiveness and resourcefulness.

### Hermes' Relationships and Roles

Hermes' significance is further amplified by his relationship with several prominent deities. As the son of Zeus, he once traveled with his father, seeking hospitality among mortals—a journey that resulted in pivotal mythological events. During this journey, Hermes and Zeus were offered refuge by the kind Baucis and Philemon, and in return, transformed their humble dwelling into a grand temple as a divine reward for their kindness.

In another notable story, Hermes stole cattle from Apollo but later exchanged them back after giving Apollo the lyre he had invented. This exchange established a complex relationship between Hermes and Apollo, showcasing Hermes' mischievous and opportunistic nature.

### Hermes' Role Among Gods and Mortals

Hermes' role extended beyond just being a trickster or inventor; he also played a critical part in facilitating communication and interactions among gods and mortals. His winged hat and sandals allowed him to travel swiftly, making him an ideal messenger. Throughout his life, his quick wit and ingenuity made him a vital figure in various mythological narratives.

Overall, Hermes is depicted as an intelligent and versatile figure, embodying creativity, cunning, and the ability to bridge the divine with the human world.

# Graph Visualization

In [32]:
# Import libraries
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import random
from IPython.display import HTML

In [36]:
# Load entities
G = nx.read_graphml(f"{WORKING_DIR}/graph_chunk_entity_relation.graphml")

# Create a network
net = Network(notebook = True, cdn_resources = "remote")
net.from_nx(G)

# Add colors and titles to nodes
for node in net.nodes:
  node["color"] = "#{:06x}".format(random.randint(0, 0xFFFFFF))
  if "description" in node:
    node["title"] = node["description"]

# Add titles to edges
for edge in net.edges:
  if "description" in  edge:
    edge["title"] = edge["description"]

# Save and display the netweok
net.show("graph.html")
net.save_graph("graph.html")

HTML(filename = "graph.html")

graph.html


In [38]:
# Print all node names
for node in G.nodes:
  print(node)

"ZEUS"
"SACRED OAK TREE"
"THE THRONE"
"DIONYSUS"
"HELEN"
"THE TROJAN WAR"
"ATHENA"
"MOUNT OLYMPUS"
"ARTEMIS"
"KING OF SPARTA"
"LYRE"
"POSEIDON"
"CURSE OF INSATIABLE HUNGER"
"THE FOREST"
"ALPHEUS"
"TROJAN WAR"
"HERA"
"APOLLO"
"HERMES"
"OLYMPUS"
"HESTIA"
"HADES"
"ERIS"
"ARES"
"SLAVES"
"DEMETER"
"THESSALY"
"THE GOLDEN APPLE"
"ORPHEUS"
"GOLDEN APPLE"
"APHRODITE"
"BAUCIS"
"ERYSICHTHON"
"THE UNDERWORLD"
"HEPHAESTUS"
"PARIS"
"ATHENS"
"PHILEMON"


In [41]:
# Target Zeus
# Load entities
G = nx.read_graphml(f"{WORKING_DIR}/graph_chunk_entity_relation.graphml")

# Target Zeus
target_node = '"ZEUS"'

# Get neighbors for that node
subgraph_nodes = list(G.neighbors(target_node)) + [target_node]
subgraph = G.subgraph(subgraph_nodes)

# Create a network
net = Network(notebook = True, cdn_resources = "remote")
net.from_nx(subgraph)

# Save and display the netweok
graph_name = f"{target_node}_graph.html"
net.save_graph(graph_name)
HTML(filename = graph_name)